In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
!git clone https://github.com/totalgood/nlpia.git

Cloning into 'nlpia'...
remote: Enumerating objects: 196, done.
remote: Counting objects: 100% (196/196), done.
remote: Compressing objects: 100% (118/118), done.
remote: Total 5827 (delta 98), reused 138 (delta 57), pack-reused 5631
Receiving objects: 100% (5827/5827), 124.02 MiB | 28.12 MiB/s, done.
Resolving deltas: 100% (3674/3674), done.


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
df = pd.read_csv('/content/nlpia/src/nlpia/data/moviedialog.csv')

In [0]:
df.drop(columns='Unnamed: 0', axis=1,inplace=True)

In [6]:
df.head(5)

,statement,reply
0,you're asking me out. that's so cute. what's y...,forget it.
1,"no, no, it's my fault we didn't have a proper ...",cameron.
2,"gosh, if only we could find kat a boyfriend...",let me see what i can do.
3,c'esc ma tete. this is my head,right. see? you're ready for the quiz.
4,how is our little find the wench a date plan p...,"well, there's someone i think might be"


In [7]:
df.shape

(64350, 2)

In [0]:
import tensorflow as tf
import os
from tensorflow.python.keras.layers import Layer
from tensorflow.python.keras import backend as K


class AttentionLayer(Layer):
    """
    This class implements Bahdanau attention (https://arxiv.org/pdf/1409.0473.pdf).
    There are three sets of weights introduced W_a, U_a, and V_a
     """

    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        # Create a trainable weight variable for this layer.

        self.W_a = self.add_weight(name='W_a',
                                   shape=tf.TensorShape((input_shape[0][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.U_a = self.add_weight(name='U_a',
                                   shape=tf.TensorShape((input_shape[1][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.V_a = self.add_weight(name='V_a',
                                   shape=tf.TensorShape((input_shape[0][2], 1)),
                                   initializer='uniform',
                                   trainable=True)

        super(AttentionLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs, verbose=False):
        """
        inputs: [encoder_output_sequence, decoder_output_sequence]
        """
        assert type(inputs) == list
        encoder_out_seq, decoder_out_seq = inputs
        if verbose:
            print('encoder_out_seq>', encoder_out_seq.shape)
            print('decoder_out_seq>', decoder_out_seq.shape)

        def energy_step(inputs, states):
            """ Step function for computing energy for a single decoder state """

            assert_msg = "States must be a list. However states {} is of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            """ Some parameters required for shaping tensors"""
            en_seq_len, en_hidden = encoder_out_seq.shape[1], encoder_out_seq.shape[2]
            de_hidden = inputs.shape[-1]

            """ Computing S.Wa where S=[s0, s1, ..., si]"""
            # <= batch_size*en_seq_len, latent_dim
            reshaped_enc_outputs = K.reshape(encoder_out_seq, (-1, en_hidden))
            # <= batch_size*en_seq_len, latent_dim
            W_a_dot_s = K.reshape(K.dot(reshaped_enc_outputs, self.W_a), (-1, en_seq_len, en_hidden))
            if verbose:
                print('wa.s>',W_a_dot_s.shape)

            """ Computing hj.Ua """
            U_a_dot_h = K.expand_dims(K.dot(inputs, self.U_a), 1)  # <= batch_size, 1, latent_dim
            if verbose:
                print('Ua.h>',U_a_dot_h.shape)

            """ tanh(S.Wa + hj.Ua) """
            # <= batch_size*en_seq_len, latent_dim
            reshaped_Ws_plus_Uh = K.tanh(K.reshape(W_a_dot_s + U_a_dot_h, (-1, en_hidden)))
            if verbose:
                print('Ws+Uh>', reshaped_Ws_plus_Uh.shape)

            """ softmax(va.tanh(S.Wa + hj.Ua)) """
            # <= batch_size, en_seq_len
            e_i = K.reshape(K.dot(reshaped_Ws_plus_Uh, self.V_a), (-1, en_seq_len))
            # <= batch_size, en_seq_len
            e_i = K.softmax(e_i)

            if verbose:
                print('ei>', e_i.shape)

            return e_i, [e_i]

        def context_step(inputs, states):
            """ Step function for computing ci using ei """
            # <= batch_size, hidden_size
            c_i = K.sum(encoder_out_seq * K.expand_dims(inputs, -1), axis=1)
            if verbose:
                print('ci>', c_i.shape)
            return c_i, [c_i]

        def create_inital_state(inputs, hidden_size):
            # We are not using initial states, but need to pass something to K.rnn funciton
            fake_state = K.zeros_like(inputs)  # <= (batch_size, enc_seq_len, latent_dim
            fake_state = K.sum(fake_state, axis=[1, 2])  # <= (batch_size)
            fake_state = K.expand_dims(fake_state)  # <= (batch_size, 1)
            fake_state = K.tile(fake_state, [1, hidden_size])  # <= (batch_size, latent_dim
            return fake_state

        fake_state_c = create_inital_state(encoder_out_seq, encoder_out_seq.shape[-1])
        fake_state_e = create_inital_state(encoder_out_seq, encoder_out_seq.shape[1])  # <= (batch_size, enc_seq_len, latent_dim

        """ Computing energy outputs """
        # e_outputs => (batch_size, de_seq_len, en_seq_len)
        last_out, e_outputs, _ = K.rnn(
            energy_step, decoder_out_seq, [fake_state_e],
        )

        """ Computing context vectors """
        last_out, c_outputs, _ = K.rnn(
            context_step, e_outputs, [fake_state_c],
        )

        return c_outputs, e_outputs

    def compute_output_shape(self, input_shape):
        """ Outputs produced by the layer """
        return [
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[1][2])),
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[0][1]))
        ]

# Preprocessing

In [0]:
contraction_mapping = {"ain't": "is not",
                       "aren't": "are not",
                       "can't": "cannot",
                       "'cause": "because",
                       "could've": "could have",
                       "couldn't": "could not",
                      "didn't": "did not",
                       "doesn't": "does not",
                       "don't": "do not",
                      "hadn't": "had not",
                       "hasn't": "has not",
                       "haven't": "have not",
                        "he'd": "he would",
                       "he'll": "he will",
                       "he's": "he is",
                        "how'd": "how did",
                        "how'd'y": "how do you",
                        "how'll": "how will",
                        "how's": "how is",
                        "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

In [19]:
data_input = [s.lower() for s in df['statement']]
data_output = [s.lower() for s in df['reply']]

print(data_input[0:3])
print(data_output[0:3])

["you're asking me out. that's so cute. what's your name again?", "no, no, it's my fault we didn't have a proper introduction ", 'gosh, if only we could find kat a boyfriend...']
['forget it.', 'cameron.', 'let me see what i can do.']


In [0]:
cleaned_text = []
for w in data_input:
  newstring = ' '.join([contraction_mapping[c] if c in contraction_mapping else c for c in w.split(" ")])
  cleaned_text.append(newstring)


In [21]:
cleaned_text[0:3]

['you are asking me out. that is so cute. what is your name again?',
 'no, no, it is my fault we did not have a proper introduction ',
 'gosh, if only we could find kat a boyfriend...']

In [0]:
cleaned_summary = []
for w in data_output:
  newstring = ' '.join([contraction_mapping[c] if c in contraction_mapping else c for c in w.split(" ")])
  tokens=newstring.split()
  newstring=''
  for char in tokens:
    if len(char) > 1:
      newstring = newstring+char+' '
  cleaned_summary.append(newstring)

In [23]:
cleaned_summary[0:3]

['forget it. ', 'cameron. ', 'let me see what can do. ']

In [0]:
df['cleaned_text']=cleaned_text
df['cleaned_summary']=cleaned_summary
df['cleaned_summary'].replace('', np.nan, inplace=True)
df.dropna(axis=0,inplace=True)

In [0]:
df.shape

In [25]:
data_inputs = [s.lower() for s in df['cleaned_text']]
data_outputs = [s.lower() for s in df['cleaned_summary']]

data_size = len(df)

split_index = int(data_size*80/100)
# print(split_index)
# validation_split_index = int(data_size*70/100)

# We will use the first 0-60th %-tile (60%) of data for the training
training_input  = data_inputs[:split_index]
training_output = data_outputs[:split_index]

# # We will use the first 60-70th %-tile (10%) of data for the training
validation_input = data_inputs[split_index:]
validation_output = data_outputs[split_index:]

print(len(training_input))
print(len(validation_input))

51428
12857


In [26]:
training_input[0:3]

['you are asking me out. that is so cute. what is your name again?',
 'no, no, it is my fault we did not have a proper introduction ',
 'gosh, if only we could find kat a boyfriend...']

In [27]:
training_output[0:3]

['forget it. ', 'cameron. ', 'let me see what can do. ']

In [59]:
START_CHAR_CODE = 1

def encode_characters(titles):
    count = 2
    encoding = {}
    decoding = {1: 'START'}
    for c in set([c for title in titles for c in title]):
        encoding[c] = count
        decoding[count] = c
        count += 1
    return encoding, decoding, count


statement_encoding, statement_decoding, statement_dict_size = encode_characters(data_inputs)
reply_encoding, reply_decoding, reply_dict_size = encode_characters(data_outputs)

print('statement character dict size:', statement_dict_size)
print('reply character dict size:', reply_dict_size)

print(list(statement_encoding.items())[0:5])
print(list(reply_encoding.items())[0:5])



statement character dict size: 46
reply character dict size: 46
[('p', 2), (';', 3), ('3', 4), ('1', 5), ('o', 6)]
[('p', 2), (';', 3), ('3', 4), ('1', 5), ('o', 6)]


In [61]:
def transform(encoding, data, vector_size):
    transformed_data = np.zeros(shape=(len(data), vector_size), dtype='int')
    for i in range(len(data)):
        for j in range(min(len(data[i]), vector_size)):
            transformed_data[i][j] = encoding[data[i][j]]
    return transformed_data

INPUT_LENGTH = 80
OUTPUT_LENGTH = 80

encoded_training_input = transform(statement_encoding, training_input, vector_size=INPUT_LENGTH)
encoded_training_output = transform(reply_encoding, training_output, vector_size=OUTPUT_LENGTH)
encoded_validation_input = transform(statement_encoding, validation_input, vector_size=INPUT_LENGTH)
encoded_validation_output = transform(reply_encoding, validation_output, vector_size=OUTPUT_LENGTH)

print('input', encoded_training_input)
print('output', encoded_training_output)

input [[10  6 44 ...  0  0  0]
 [31  6 13 ...  0  0  0]
 [34  6 20 ...  0  0  0]
 ...
 [29 35 10 ...  0  0  0]
 [32 44 41 ...  0  0  0]
 [10  6 44 ...  0  0  0]]
output [[11  6 17 ...  0  0  0]
 [37 34 12 ...  0  0  0]
 [32 28  7 ...  0  0  0]
 ...
 [35 34 15 ...  0  0  0]
 [35 18 13 ...  0  0  0]
 [10 28 34 ...  0  0  0]]


In [0]:
from __future__ import print_function

import os
import pandas as pd
import numpy as np

from tensorflow.keras.layers import Input, Embedding, LSTM, TimeDistributed, Dense
from tensorflow.keras.models import Model, load_model

from tensorflow.keras.layers import Activation, dot, concatenate

In [0]:
encoder_input = Input(shape=(INPUT_LENGTH,))
decoder_input = Input(shape=(OUTPUT_LENGTH,))

In [66]:
# Encoder
# encoder = Embedding(statement_dict_size, 64, input_length=INPUT_LENGTH, mask_zero=True)(encoder_input)
# encoder = LSTM(64)(encoder)

# print(encoder.get_shape())

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
(?, 64)


In [68]:
# decoder = Embedding(reply_dict_size, 64, input_length=OUTPUT_LENGTH, mask_zero=True)(decoder_input)

# decoder = LSTM(64, return_sequences=True)(decoder, initial_state=[encoder, encoder])
# decoder = TimeDistributed(Dense(reply_dict_size, activation="softmax"))(decoder)

# print(decoder.get_shape())

(?, 80, 46)


In [69]:
encoder = Embedding(statement_dict_size, 64, input_length=INPUT_LENGTH, mask_zero=True)(encoder_input)
encoder_output, state_h, state_c = LSTM(64, return_sequences=True, return_state=True, unroll=True)(encoder)
# encoder = LSTM(64, return_sequences=True, unroll=True)(encoder)
# encoder_last = encoder[:,-1,:]

# print('encoder', encoder)
# print('encoder_last', encoder_last)

decoder = Embedding(reply_dict_size, 64, input_length=OUTPUT_LENGTH, mask_zero=True)(decoder_input)
decoder_output,decoder_fwd_state,decoder_back_state = LSTM(64, return_sequences=True, return_state=True, unroll=True)(decoder, initial_state=[state_h, state_c])
# decoder = LSTM(64, return_sequences=True, unroll=True)(decoder, initial_state=[encoder_last, encoder_last])

print('decoder', decoder)

decoder Tensor("embedding_4/embedding_lookup/Identity_1:0", shape=(?, 80, 64), dtype=float32)


In [70]:
# Equation (7) with 'dot' score from Section 3.1 in the paper.
# Note that we reuse Softmax-activation layer instead of writing tensor calculation
attention = dot([decoder, encoder], axes=[2, 2])
attention = Activation('softmax', name='attention')(attention)
print('attention', attention)

context = dot([attention, encoder], axes=[2,1])
print('context', context)

decoder_combined_context = concatenate([context, decoder])
print('decoder_combined_context', decoder_combined_context)

# Has another weight + tanh layer as described in equation (5) of the paper
output = TimeDistributed(Dense(64, activation="tanh"))(decoder_combined_context)
output = TimeDistributed(Dense(reply_dict_size, activation="softmax"))(output)
print('output', output)

attention Tensor("attention/truediv:0", shape=(?, 80, 80), dtype=float32)
context Tensor("dot_1/MatMul:0", shape=(?, 80, 64), dtype=float32)
decoder_combined_context Tensor("concatenate/concat:0", shape=(?, 80, 128), dtype=float32)
output Tensor("time_distributed_2/Reshape_1:0", shape=(?, 80, 46), dtype=float32)


In [71]:
model = Model(inputs=[encoder_input, decoder_input], outputs=[output])
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 80)]         0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 80)]         0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 80, 64)       2944        input_2[0][0]                    
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 80, 64)       2944        input_1[0][0]                    
______________________________________________________________________________________________

In [0]:
training_encoder_input = encoded_training_input
training_decoder_input = np.zeros_like(encoded_training_output)
training_decoder_input[:, 1:] = encoded_training_output[:,:-1]
training_decoder_input[:, 0] = START_CHAR_CODE
training_decoder_output = np.eye(reply_dict_size)[encoded_training_output.astype('int')]

validation_encoder_input = encoded_validation_input
validation_decoder_input = np.zeros_like(encoded_validation_output)
validation_decoder_input[:, 1:] = encoded_validation_output[:,:-1]
validation_decoder_input[:, 0] = START_CHAR_CODE
validation_decoder_output = np.eye(reply_dict_size)[encoded_validation_output.astype('int')]

In [75]:
if os.path.isfile('model.h5'):
    model = load_model('model.h5')
else:
    model.fit(x=[training_encoder_input, training_decoder_input], y=[training_decoder_output],
          validation_data=([validation_encoder_input, validation_decoder_input], [validation_decoder_output]),
          verbose=2, batch_size=64, epochs=20)

# model.fit(x=[training_encoder_input, training_decoder_input], y=[training_decoder_output],
#           validation_data=([validation_encoder_input, validation_decoder_input], [validation_decoder_output]),
#           verbose=2, batch_size=64, epochs=5)

model.save('model.h5')

Train on 51428 samples, validate on 12857 samples
Epoch 1/20
51428/51428 - 13s - loss: 1.0807 - val_loss: 1.0584
Epoch 2/20
51428/51428 - 8s - loss: 1.0259 - val_loss: 1.0548
Epoch 3/20
51428/51428 - 8s - loss: 1.0238 - val_loss: 1.0534
Epoch 4/20
51428/51428 - 8s - loss: 1.0230 - val_loss: 1.0528
Epoch 5/20
51428/51428 - 8s - loss: 1.0225 - val_loss: 1.0526
Epoch 6/20
51428/51428 - 8s - loss: 1.0222 - val_loss: 1.0522
Epoch 7/20
51428/51428 - 8s - loss: 1.0220 - val_loss: 1.0520
Epoch 8/20
51428/51428 - 8s - loss: 1.0218 - val_loss: 1.0520
Epoch 9/20
51428/51428 - 8s - loss: 1.0216 - val_loss: 1.0521
Epoch 10/20
51428/51428 - 8s - loss: 1.0215 - val_loss: 1.0519
Epoch 11/20
51428/51428 - 8s - loss: 1.0214 - val_loss: 1.0518
Epoch 12/20
51428/51428 - 8s - loss: 1.0213 - val_loss: 1.0516
Epoch 13/20
51428/51428 - 8s - loss: 1.0211 - val_loss: 1.0515
Epoch 14/20
51428/51428 - 8s - loss: 1.0210 - val_loss: 1.0515
Epoch 15/20
51428/51428 - 8s - loss: 1.0209 - val_loss: 1.0513
Epoch 16/20
5

In [0]:
def generate_katakana_sequence(text):
    encoder_input = transform(statement_encoding, [text.lower()], 80)
    decoder_input = np.zeros(shape=(len(encoder_input), OUTPUT_LENGTH))
    decoder_input[:,0] = START_CHAR_CODE
    for i in range(1, OUTPUT_LENGTH):
        output = model.predict([encoder_input, decoder_input]).argmax(axis=2)
        decoder_input[:,i] = output[:,i]
    return decoder_input[0, 1:]

def decode(decoding, sequence):
    text = ''
    for i in sequence:
        if i == 0:
            break
        text += decoding[i]
    return text

def to_katakana(text):
    decoder_output = generate_katakana_sequence(text)
    return decode(reply_decoding, decoder_output)

In [86]:
to_katakana('Banana')

'                                                                               '

# Below pieces are part of some other notebook. Ignore

In [0]:
input_texts, target_texts = [], []
input_vocabulary = set()
output_vocabulary = set()
start_token = '\t'
stop_token = '\n'
max_training_samples = min(25000, len(df) -1)

In [0]:
for input_text, target_text in zip(df.statement, df.reply):
  target_text = start_token + target_text + stop_token
  input_texts.append(input_text)
  target_texts.append(target_text)
  for char in input_text:
    input_vocabulary.add(char)
  for char in target_text:
    output_vocabulary.add(char) 

In [0]:
input_vocabulary = sorted(input_vocabulary)
output_vocabulary = sorted(output_vocabulary)

In [0]:
input_vocab_size = len(input_vocabulary)
output_vocab_size = len(output_vocabulary)

In [0]:
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [0]:
input_char_index = dict((c,i) for i,c in enumerate(input_vocabulary))
target_char_index = dict((c,i) for i,c in enumerate(output_vocabulary))

reverse_input_char_index = dict((i, c) for c, i in input_char_index.items())
reverse_target_char_index = dict((i, c) for c, i in target_char_index.items())

In [0]:
import numpy as np
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, input_vocab_size), dtype = 'float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, output_vocab_size), dtype = 'float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, output_vocab_size), dtype='float32')

In [0]:
encoder_input_data.reshape()

(100, 44)

In [0]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
  for t,c in enumerate(input_text):
    encoder_input_data[i, t, input_char_index[c]] = 1.
  for t,c in enumerate(target_text):
    decoder_input_data[i, t, target_char_index[c]] = 1.
    if t > 0:
      decoder_target_data[i, t-1, target_char_index[c]] = 1.

In [0]:
import numpy as np  
import pandas as pd 
import re           
from bs4 import BeautifulSoup 
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords  
import nltk
nltk.download('stopwords') 
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional, GRU
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
encoder_inputs.shape

TensorShape([Dimension(None), Dimension(None), Dimension(100), Dimension(44)])

In [0]:
enc_emb.shape

TensorShape([Dimension(None), Dimension(44), Dimension(512)])

In [0]:
encoder_input_data.shape

(64350, 100, 44)

In [0]:
enc_emb.shape

TensorShape([Dimension(None), Dimension(44), Dimension(512)])

In [0]:
input_vocab_size

44

In [0]:
max_decoder_seq_length

102

In [0]:
encoder_inputs.shape

TensorShape([Dimension(64), Dimension(100), Dimension(44)])

In [0]:
decoder_inputs.shape

TensorShape([Dimension(64), Dimension(102), Dimension(46)])

In [0]:
encoder_outputs.shape

TensorShape([Dimension(64), Dimension(100), Dimension(512)])

In [0]:
decoder_outputs.shape

TensorShape([Dimension(64), Dimension(102), Dimension(512)])

In [0]:
attn_out.shape

TensorShape([Dimension(64), Dimension(None), Dimension(512)])

In [0]:
attn_out.shape

TensorShape([Dimension(64), Dimension(None), Dimension(512)])

In [0]:
output_vocab_size

46

In [0]:
# from keras.models import Model
# from keras.layers import Input, LSTM, Dense
from keras import backend as K 
K.clear_session() 
batch_size = 64
epochs = 100
num_neurons = 512

encoder_inputs = Input(batch_shape=(batch_size, max_encoder_seq_length, input_vocab_size))
decoder_inputs = Input(batch_shape=(batch_size, max_decoder_seq_length, output_vocab_size))

encoder_lstm1 = LSTM(num_neurons, return_sequences=True, return_state=True, name='encoder_lstm1')
encoder_outputs, state_h, state_c = encoder_lstm1(encoder_inputs)

decoder_lstm1 = LSTM(num_neurons, return_sequences=True, return_state=True, name='decoder_lstm1')
decoder_outputs,decoder_fwd_state,decoder_back_state = decoder_lstm1(decoder_inputs,initial_state=[state_h, state_c]) 

attn_layer = AttentionLayer(name='attention_layer') 
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])

# Concat attention output and decoder LSTM output 
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

#Dense layer
dense = Dense(output_vocab_size, activation='softmax', name='softmax_layer')
decoder_time = TimeDistributed(dense, name = 'time_distributed_layer') 
decoder_pred = decoder_time(decoder_concat_input) 

# Define the model
model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=decoder_pred) 
model.summary()



Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(64, 100, 44)]      0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(64, 102, 46)]      0                                            
__________________________________________________________________________________________________
encoder_lstm1 (LSTM)            [(64, 100, 512), (64 1140736     input_1[0][0]                    
__________________________________________________________________________________________________
decoder_lstm1 (LSTM)            [(64, 102, 512), (64 1144832     input_2[0][0]                    
                                                                 encoder_lstm1[0][1]          

In [0]:
encoder_inputs.shape

TensorShape([Dimension(None), Dimension(100), Dimension(44)])

In [0]:
enc_emb.shape

TensorShape([Dimension(None), Dimension(100), Dimension(512)])

In [0]:
encoder_outputs.shape

TensorShape([Dimension(None), Dimension(100), Dimension(512)])

In [0]:
decoder_inputs.shape

TensorShape([Dimension(None), Dimension(None), Dimension(None)])

In [0]:
dec_emb.shape

TensorShape([Dimension(None), Dimension(None), Dimension(512)])

In [0]:
decoder_inputs.shape

TensorShape([Dimension(None), Dimension(None), Dimension(46)])

In [0]:
decoder_outputs.shape

TensorShape([Dimension(None), Dimension(None), Dimension(512)])

In [0]:
attn_out.shape

TensorShape([Dimension(None), Dimension(None), Dimension(512)])

In [0]:
# from keras.models import Model
# from keras.layers import Input, LSTM, Dense
from keras import backend as K 
K.clear_session() 
batch_size = 64
epochs = 100
num_neurons = 512


# Encoder 
encoder_inputs = Input(shape=(max_encoder_seq_length,input_vocab_size,)) 
# enc_emb = Embedding(input_vocab_size, num_neurons,trainable=True)(encoder_inputs) 

#LSTM 1 
encoder_lstm1 = LSTM(num_neurons,return_sequences=True,return_state=True) 
encoder_outputs, state_h, state_c = encoder_lstm1(encoder_inputs) 

# #LSTM 2 
# encoder_lstm2 = LSTM(num_neurons,return_sequences=True,return_state=True) 
# encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1) 

# #LSTM 3 
# encoder_lstm3=LSTM(num_neurons, return_state=True, return_sequences=True) 
# encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2) 

# Set up the decoder. 
decoder_inputs = Input(shape=(max_decoder_seq_length, output_vocab_size)) 
# dec_emb_layer = Embedding(output_vocab_size, num_neurons, trainable=True) 
# dec_emb = dec_emb_layer(decoder_inputs) 

#LSTM using encoder_states as initial state
decoder_lstm = LSTM(num_neurons, return_sequences=True, return_state=True) 
decoder_outputs,decoder_fwd_state, decoder_back_state = decoder_lstm(decoder_inputs,initial_state=[state_h, state_c]) 

#Attention Layer
attn_layer = AttentionLayer(name='attention_layer') 
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs]) 

# Concat attention output and decoder LSTM output 
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

#Dense layer
decoder_dense = TimeDistributed(Dense(output_vocab_size, activation='softmax')) 
decoder_pred = decoder_dense(decoder_concat_input) 

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_pred) 
model.summary()




ValueError: ignored

In [0]:
model = Model([encoder_inputs, decoder_inputs], decoder_pred)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size = batch_size, epochs = epochs, validation_split=0.1)

Train on 57915 samples, validate on 6435 samples
Epoch 1/100


InvalidArgumentError: ignored

In [0]:
from keras import backend as K 
K.clear_session() 
batch_size = 64
epochs = 100
num_neurons = 512

encoder_inputs = Input(batch_shape=(batch_size, max_encoder_seq_length, input_vocab_size))
decoder_inputs = Input(batch_shape=(batch_size, max_decoder_seq_length, output_vocab_size))

encoder_gru = GRU(num_neurons, return_sequences=True, return_state=True, name='encoder_gru')
encoder_outputs, encoder_state = encoder_gru(encoder_inputs)

decoder_gru = GRU(num_neurons, return_sequences=True, return_state=True, name='decoder_gru')
decoder_outputs,decoder_state = decoder_gru(decoder_inputs,initial_state= encoder_state) 

attn_layer = AttentionLayer(name='attention_layer') 
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])

# Concat attention output and decoder LSTM output 
decoder_concat_input = Concatenate(axis=1, name='concat_layer')([decoder_outputs, attn_out])

#Dense layer
dense = Dense(output_vocab_size, activation='softmax', name='softmax_layer')
decoder_time = TimeDistributed(dense, name = 'time_distributed_layer') 
decoder_pred = decoder_time(decoder_concat_input) 

# Define the model
model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=decoder_pred) 
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(64, 100, 44)]      0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(64, 102, 46)]      0                                            
__________________________________________________________________________________________________
encoder_gru (GRU)               [(64, 100, 512), (64 855552      input_1[0][0]                    
__________________________________________________________________________________________________
decoder_gru (GRU)               [(64, 102, 512), (64 858624      input_2[0][0]                    
                                                                 encoder_gru[0][1]            

In [0]:
encoder_input_data.shape

(64350, 100, 44)

In [0]:
# Assemble the model for sequence generation
encoder_model = Model(encoder_inputs, encoder_states)
thought_input = [Input(shape=(num_neurons,)), Input(shape=(num_neurons,))]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=thought_input)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(inputs=[decoder_inputs] + thought_input,
                     outputs=[decoder_outputs] + decoder_states)

In [0]:
# Build a character-based translator 
def decode_sequence(input_seq):
  thought = encoder_model.predict(input_seq)
  
  target_seq = np.zeros((1,1,output_vocab_size))
  target_seq[0, 0, target_token_index[start_token]] = 1.
  stop_condition = False
  generated_sequence = ''
  
  while not stop_condition:
    output_tokens, h, c = decoder_model.predict([target_seq] + thought)
    generated_token_idx = np.argmax(output_tokens[0, -1, :])
    generated_char = reverse_target_char_index[generated_token_idx]
    generated_sequence += generated_char
    if (generated_char == stop_token or len(generated_sequence) > max_decoder_seq_length):
      stop_condition = True
    target_seq = np.zeros((1, 1, output_vocab_size))
    target_seq[0, 0, generated_token_idx] = 1.
    thought = [h, c]
  return generated_sequence

In [0]:
# Generating a response
def response(input_text):
  input_seq = np.zeros((1, max_encoder_seq_length, input_vocab_size), dtype='float32')
  for t, char in enumerate(input_text):
    input_seq[0, t, input_token_index[char]] = 1.
  decoded_sentence = decode_sequence(input_seq)
  print('Bot Reply (Decoded sentence):', decoded_sentence)